## Nutritional Analysis of Fast Food Menus

Code to analyze the nutritional content of fast food menus and create meal options based on nutritional goals.

In [45]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib as plt
%matplotlib inline

import itertools as it
import re

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Import menu item data

In [46]:
df_menu = pd.read_csv(r'C:\Users\Jenny\menu_nf.csv', index_col=0)
df_menu = df_menu.drop(['dietary_fiber','brand_id', 'last_updated','serving_size_qty','serving_size_unit', 'serving_weight_grams', 'servings_per_container'], axis=1)
df_menu = df_menu[['brand_name',"item_name", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "item_type", "tag"]]
df_menu.sort_values('item_name').head()

,brand_name,item_name,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,item_type,tag
35,Burger King,1% Low Fat Chocolate Milk,160.0,2.5,26.0,25.0,8.0,1.5,15.0,150.0,drink,none
172,McDonald's,1% Low Fat Milk Jug,100.0,2.5,12.0,12.0,8.0,1.5,10.0,125.0,drink,none
376,McDonald's,1% Low Fat Milk Jug California,130.0,2.5,16.0,15.0,11.0,1.5,15.0,160.0,drink,none
69,Wendy's,1/4 lb. Hamburger Patty,240.0,18.0,0.0,0.0,20.0,7.0,75.0,280.0,entree,none
67,Wendy's,10-Piece Chicken Nuggets,420.0,27.0,24.0,0.0,22.0,5.0,80.0,850.0,entree,none


In [47]:
# Average macronutrient content for each item type
df_menu.groupby(['item_type','brand_name'])[['calories','total_fat','carbs','sugar','protein']].mean().round()

calories  total_fat  carbs  sugar  protein
item_type brand_name                                             
dessert   Burger King  380.0     13.0       59.0   44.0   7.0    
          McDonald's   378.0     13.0       50.0   41.0   7.0    
          Taco Bell    370.0     20.0       44.0   23.0   4.0    
          Wendy's      395.0     10.0       66.0   55.0   11.0   
drink     Burger King  246.0     1.0        62.0   60.0   1.0    
          McDonald's   185.0     2.0        42.0   40.0   2.0    
          Taco Bell    195.0     0.0        42.0   41.0   0.0    
          Wendy's      161.0     1.0        41.0   40.0   0.0    
entree    Burger King  541.0     32.0       40.0   8.0    25.0   
          McDonald's   533.0     26.0       48.0   9.0    26.0   
          Taco Bell    417.0     19.0       42.0   3.0    16.0   
          Wendy's      417.0     22.0       31.0   7.0    24.0   
misc      Burger King  104.0     9.0        7.0    5.0    0.0    
          McDonald's   64.0      3.0        9.0    6.0    0.0    
          Taco Bell    37.0      4.0        1.0    1.0    0.0    
          Wendy's      89.0      4.0        9.0    2.0    4.0    
side      Burger King  368.0     20.0       41.0   2.0    6.0    
          McDonald's   289.0     13.0       35.0   4.0    8.0    
          Taco Bell    278.0     14.0       32.0   2.0    6.0    
          Wendy's      356.0     15.0       45.0   5.0    10.0

### Create a dataframe with entree-only meal options that meet nutritional goals

In [48]:
# Set daily nutritional goals. Calculate macronutrients as a percentage of calories according to dietary guidelines.
goal_daily = {}
def goal_calc(calories):
    total_fat = round(calories * 0.3 / 9.0)
    carbs = round(calories * 0.6 / 4.0)
    sugar = round(calories * 0.05 / 4.0)
    goal_daily['calories'] = calories
    goal_daily['total_fat'] = total_fat
    goal_daily['carbs'] = carbs
    goal_daily['sugar'] = sugar

calories = pd.Series([1500.0, 2000.0, 2500.0, 3000.0])
goal_calc(calories)
df_goal_daily = pd.DataFrame(goal_daily)
df_goal_daily

,calories,carbs,sugar,total_fat
0,1500.0,225.0,19.0,50.0
1,2000.0,300.0,25.0,67.0
2,2500.0,375.0,31.0,83.0
3,3000.0,450.0,38.0,100.0


In [49]:
# Set this meal to be 40% of daily goal
meal_portion = 0.4

df_goals = df_goal_daily * meal_portion
df_goals

,calories,carbs,sugar,total_fat
0,600.0,90.0,7.6,20.0
1,800.0,120.0,10.0,26.8
2,1000.0,150.0,12.4,33.2
3,1200.0,180.0,15.2,40.0


In [50]:
# Define item_types and nutritional goals
entree = df_menu['item_type'] == 'entree'
side = df_menu['item_type'] == 'side'
dessert = df_menu['item_type'] == 'dessert'
drink = df_menu['item_type'] == 'drink'
coffee = df_menu['tag'] == 'coffee'
soda = df_menu['tag'] == 'soda'

within_goal_0 = (df_menu['calories'] < df_goals.loc[0,'calories']) & (df_menu['carbs'] < df_goals.loc[0,'carbs']) & (df_menu['total_fat'] < df_goals.loc[0,'total_fat']) & (df_menu['sugar'] < df_goals.loc[0,'sugar'])
within_goal_1 = (df_menu['calories'] < df_goals.loc[1,'calories']) & (df_menu['carbs'] < df_goals.loc[1,'carbs']) & (df_menu['total_fat'] < df_goals.loc[1,'total_fat']) & (df_menu['sugar'] < df_goals.loc[1,'sugar'])
within_goal_2 = (df_menu['calories'] < df_goals.loc[2,'calories']) & (df_menu['carbs'] < df_goals.loc[2,'carbs']) & (df_menu['total_fat'] < df_goals.loc[2,'total_fat']) & (df_menu['sugar'] < df_goals.loc[2,'sugar'])
within_goal_3 = (df_menu['calories'] < df_goals.loc[3,'calories']) & (df_menu['carbs'] < df_goals.loc[3,'carbs']) & (df_menu['total_fat'] < df_goals.loc[3,'total_fat']) & (df_menu['sugar'] < df_goals.loc[3,'sugar'])

# Create a dataframe with all entrees that meet nutritional goals for each calorie scenario
df_meal_0 = df_menu.loc[entree & within_goal_0]
df_meal_0['meal_type'] = 'entree_only'
df_meal_0 = df_meal_0.rename(columns = {'item_name':'entree'})

df_meal_1 = df_menu.loc[entree & within_goal_1]
df_meal_1['meal_type'] = 'entree_only'
df_meal_1 = df_meal_1.rename(columns = {'item_name':'entree'})

df_meal_2 = df_menu.loc[entree & within_goal_2]
df_meal_2['meal_type'] = 'entree_only'
df_meal_2 = df_meal_2.rename(columns = {'item_name':'entree'})

df_meal_3 = df_menu.loc[entree & within_goal_3]
df_meal_3['meal_type'] = 'entree_only'
df_meal_3 = df_meal_3.rename(columns = {'item_name':'entree'})

C:\Users\Jenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Jenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Jenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

### Add entree-side combinations that meet nutritional goals

In [51]:
# Create a dictionary for each nutritional fact (e.g., dict_calories), then add that nutritional fact to df_entree_side.

key_calories = list(df_menu['item_name'])
value_calories = list(df_menu['calories'])
dict_calories = dict(zip(key_calories, value_calories))

key_total_fat = list(df_menu['item_name'])
value_total_fat = list(df_menu['total_fat'])
dict_total_fat = dict(zip(key_total_fat, value_total_fat))

key_carbs = list(df_menu['item_name'])
value_carbs = list(df_menu['carbs'])
dict_carbs = dict(zip(key_carbs, value_carbs))

key_sugar = list(df_menu['item_name'])
value_sugar = list(df_menu['sugar'])
dict_sugar = dict(zip(key_sugar, value_sugar))

key_protein = list(df_menu['item_name'])
value_protein = list(df_menu['protein'])
dict_protein = dict(zip(key_protein, value_protein))

key_saturated_fat = list(df_menu['item_name'])
value_saturated_fat = list(df_menu['saturated_fat'])
dict_saturated_fat = dict(zip(key_saturated_fat, value_saturated_fat))

key_cholesterol = list(df_menu['item_name'])
value_cholesterol = list(df_menu['cholesterol'])
dict_cholesterol = dict(zip(key_cholesterol, value_cholesterol))

key_sodium = list(df_menu['item_name'])
value_sodium = list(df_menu['sodium'])
dict_sodium = dict(zip(key_sodium, value_sodium))

key_tag = list(df_menu['item_name'])
value_tag = list(df_menu['tag'])
dict_tag = dict(zip(key_tag, value_tag))

key_brand_name = list(df_menu['item_name'])
value_brand_name = list(df_menu['brand_name'])
dict_brand_name = dict(zip(key_brand_name, value_brand_name))

In [52]:
# Create df_entree_side with all possible entree and side combinations
list_entree_side = list(it.product(df_menu.loc[(df_menu['brand_name'] == "McDonald's") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "McDonald's") & (df_menu['item_type'] == 'side')]['item_name']))
list_entree_side.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Wendy's") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Wendy's") & (df_menu['item_type'] == 'side')]['item_name'])))
list_entree_side.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Burger King") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Burger King") & (df_menu['item_type'] == 'side')]['item_name'])))
list_entree_side.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Taco Bell") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Taco Bell") & (df_menu['item_type'] == 'side')]['item_name'])))

df_entree_side = pd.DataFrame(list_entree_side, columns = ['entree','side'])
df_entree_side

,entree,side
0,Chicken Selects (3 Piece),Small French Fries
1,Chicken Selects (3 Piece),Mac N Cheese
2,Chicken Selects (3 Piece),Cuties
3,Chicken Selects (3 Piece),Mozzarella Sticks (3 Piece)
4,Chicken Selects (3 Piece),Medium French Fries
5,Chicken Selects (3 Piece),McCheddar Potato Bites (8 piece)
6,Chicken Selects (3 Piece),Chicken Noodle Soup
7,Chicken Selects (3 Piece),Spam Eggs and Rice
8,Chicken Selects (3 Piece),McCheddar Potato Bites (12 piece)
9,Chicken Selects (3 Piece),Chicken McBites


In [53]:
# Add nutritional data to df_entree_side

for row in df_entree_side:
    entree_name = df_entree_side['entree']
    side_name = df_entree_side['side']
    df_entree_side['calories'] = entree_name.map(dict_calories) + side_name.map(dict_calories)
    df_entree_side['total_fat'] = entree_name.map(dict_total_fat) + side_name.map(dict_total_fat)
    df_entree_side['carbs'] = entree_name.map(dict_carbs) + side_name.map(dict_carbs)
    df_entree_side['sugar'] = entree_name.map(dict_sugar) + side_name.map(dict_sugar)
    df_entree_side['protein'] = entree_name.map(dict_protein) + side_name.map(dict_protein)
    df_entree_side['saturated_fat'] = entree_name.map(dict_saturated_fat) + side_name.map(dict_saturated_fat)
    df_entree_side['cholesterol'] = entree_name.map(dict_cholesterol) + side_name.map(dict_cholesterol)
    df_entree_side['sodium'] = entree_name.map(dict_sodium) + side_name.map(dict_sodium)
    df_entree_side['tag'] = entree_name.map(dict_tag) + ', ' + side_name.map(dict_tag)
    df_entree_side['brand_name'] = entree_name.map(dict_brand_name)
    
df_entree_side['meal_type'] = 'entree_side'
df_entree_side.head()

,entree,side,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,tag,brand_name,meal_type
0,Chicken Selects (3 Piece),Small French Fries,610.0,33.0,50.0,0.0,27.0,5.0,70.0,850.0,"none, none",McDonald's,entree_side
1,Chicken Selects (3 Piece),Mac N Cheese,510.0,26.0,37.0,4.0,30.0,6.0,80.0,930.0,"none, none",McDonald's,entree_side
2,Chicken Selects (3 Piece),Cuties,415.0,22.0,29.0,6.0,24.0,3.5,70.0,690.0,"none, none",McDonald's,entree_side
3,Chicken Selects (3 Piece),Mozzarella Sticks (3 Piece),570.0,32.0,39.0,1.0,32.0,7.5,90.0,1150.0,"none, none",McDonald's,entree_side
4,Chicken Selects (3 Piece),Medium French Fries,720.0,38.0,65.0,0.0,28.0,5.5,70.0,920.0,"none, none",McDonald's,entree_side


In [54]:
# Add entree-side combinations that meet nutritional goals to each df_meal scenario
within_goal_entree_side_0 = (df_entree_side['calories'] < df_goals.loc[0,'calories']) & (df_entree_side['carbs'] < df_goals.loc[0,'carbs']) & (df_entree_side['total_fat'] < df_goals.loc[0,'total_fat']) & (df_entree_side['sugar'] < df_goals.loc[0,'sugar'])
df_meal_0 = df_meal_0.append(df_entree_side.loc[within_goal_entree_side_0])
df_meal_0 = df_meal_0[['brand_name',"entree", "side", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_1 = (df_entree_side['calories'] < df_goals.loc[1,'calories']) & (df_entree_side['carbs'] < df_goals.loc[1,'carbs']) & (df_entree_side['total_fat'] < df_goals.loc[1,'total_fat']) & (df_entree_side['sugar'] < df_goals.loc[1,'sugar'])
df_meal_1 = df_meal_1.append(df_entree_side.loc[within_goal_entree_side_1])
df_meal_1 = df_meal_1[['brand_name',"entree", "side", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_2 = (df_entree_side['calories'] < df_goals.loc[2,'calories']) & (df_entree_side['carbs'] < df_goals.loc[2,'carbs']) & (df_entree_side['total_fat'] < df_goals.loc[2,'total_fat']) & (df_entree_side['sugar'] < df_goals.loc[2,'sugar'])
df_meal_2 = df_meal_2.append(df_entree_side.loc[within_goal_entree_side_2])
df_meal_2 = df_meal_2[['brand_name',"entree", "side", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_3 = (df_entree_side['calories'] < df_goals.loc[3,'calories']) & (df_entree_side['carbs'] < df_goals.loc[3,'carbs']) & (df_entree_side['total_fat'] < df_goals.loc[3,'total_fat']) & (df_entree_side['sugar'] < df_goals.loc[3,'sugar'])
df_meal_3 = df_meal_3.append(df_entree_side.loc[within_goal_entree_side_3])
df_meal_3 = df_meal_3[['brand_name',"entree", "side", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

df_meal_0

,brand_name,entree,side,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,meal_type,tag
22,McDonald's,Egg McMuffin,NaN,300.0,12.0,30.0,3.0,18.0,6.0,245.0,730.0,entree_only,breakfast
24,McDonald's,Grilled Onion Cheddar Burger,NaN,310.0,13.0,32.0,6.0,15.0,6.0,45.0,640.0,entree_only,none
34,McDonald's,McDouble,NaN,380.0,18.0,34.0,7.0,23.0,8.0,70.0,840.0,entree_only,none
38,McDonald's,"Ranch Snack Wrap, Grilled",NaN,300.0,13.0,25.0,2.0,21.0,4.5,65.0,890.0,entree_only,none
45,McDonald's,"Premium McWrap Chicken & Ranch, Grilled",NaN,490.0,19.0,41.0,5.0,39.0,7.0,120.0,1510.0,entree_only,none
49,McDonald's,Buffalo Ranch McChicken,NaN,360.0,16.0,40.0,5.0,15.0,3.0,40.0,800.0,entree_only,none
66,McDonald's,Country Ham Biscuit,NaN,340.0,14.0,36.0,2.0,15.0,8.0,35.0,1600.0,entree_only,breakfast
67,McDonald's,Biscuit Chicken Sausage,NaN,350.0,17.0,36.0,2.0,13.0,8.0,50.0,1110.0,entree_only,breakfast
84,McDonald's,Pico Guacamole Grilled Chicken Sandwich on Potato Roll,NaN,470.0,17.0,40.0,6.0,38.0,5.0,115.0,1520.0,entree_only,none
125,McDonald's,Steak & Egg Burrito,NaN,350.0,15.0,38.0,3.0,16.0,6.0,115.0,950.0,entree_only,breakfast


### Add entree_side_dessert combinations that meet nutritional goals

In [55]:
# Create df_entree_side_dessert with all possible entree, side, dessert combinations
list_entree_side_dessert = list(it.product(df_menu.loc[(df_menu['brand_name'] == "McDonald's") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "McDonald's") & (df_menu['item_type'] == 'side')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "McDonald's") & (df_menu['item_type'] == 'dessert')]['item_name']))
list_entree_side_dessert.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Wendy's") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Wendy's") & (df_menu['item_type'] == 'side')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Wendy's") & (df_menu['item_type'] == 'dessert')]['item_name'])))
list_entree_side_dessert.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Burger King") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Burger King") & (df_menu['item_type'] == 'side')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Burger King") & (df_menu['item_type'] == 'dessert')]['item_name'])))
list_entree_side_dessert.extend(list(it.product(df_menu.loc[(df_menu['brand_name'] == "Taco Bell") & (df_menu['item_type'] == 'entree')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Taco Bell") & (df_menu['item_type'] == 'side')]['item_name'], df_menu.loc[(df_menu['brand_name'] == "Taco Bell") & (df_menu['item_type'] == 'dessert')]['item_name'])))

#list_entree_side_dessert = list(it.product(df_menu.loc[df_menu['item_type'] == 'entree']['item_name'], df_menu.loc[df_menu['item_type'] == 'side']['item_name'], df_menu.loc[df_menu['item_type'] == 'drink']['item_name']))
df_entree_side_dessert = pd.DataFrame(list_entree_side_dessert, columns = ['entree','side', 'dessert'])
df_entree_side_dessert

,entree,side,dessert
0,Chicken Selects (3 Piece),Small French Fries,Strawberry & Creme Pie
1,Chicken Selects (3 Piece),Small French Fries,"Chocolate Shake, Medium"
2,Chicken Selects (3 Piece),Small French Fries,"Arctic Orange Shake, Small"
3,Chicken Selects (3 Piece),Small French Fries,Hot Fudge Sundae EOTF
4,Chicken Selects (3 Piece),Small French Fries,Cherry and Creme Pie
5,Chicken Selects (3 Piece),Small French Fries,Marionberry Pie Ice Cream
6,Chicken Selects (3 Piece),Small French Fries,Udderly Chocolate Ice Cream
7,Chicken Selects (3 Piece),Small French Fries,Chocolate Coffee Cake
8,Chicken Selects (3 Piece),Small French Fries,Mixed Berry and Lemon Creme Pie
9,Chicken Selects (3 Piece),Small French Fries,Cinnamon Coffee Cake


In [56]:
# Create df_entree_side_dessert with all possible entree and side combinations, and the total nutritional data for each combination

for row in df_entree_side_dessert:
    entree_name = df_entree_side_dessert['entree']
    side_name = df_entree_side_dessert['side']
    dessert_name = df_entree_side_dessert['dessert']
    df_entree_side_dessert['calories'] = entree_name.map(dict_calories) + side_name.map(dict_calories) + dessert_name.map(dict_calories)
    df_entree_side_dessert['total_fat'] = entree_name.map(dict_total_fat) + side_name.map(dict_total_fat) + dessert_name.map(dict_total_fat)
    df_entree_side_dessert['carbs'] = entree_name.map(dict_carbs) + side_name.map(dict_carbs) + dessert_name.map(dict_carbs)
    df_entree_side_dessert['sugar'] = entree_name.map(dict_sugar) + side_name.map(dict_sugar) + dessert_name.map(dict_sugar)
    df_entree_side_dessert['protein'] = entree_name.map(dict_protein) + side_name.map(dict_protein) + dessert_name.map(dict_protein)
    df_entree_side_dessert['saturated_fat'] = entree_name.map(dict_saturated_fat) + side_name.map(dict_saturated_fat) + dessert_name.map(dict_saturated_fat)
    df_entree_side_dessert['cholesterol'] = entree_name.map(dict_cholesterol) + side_name.map(dict_cholesterol) + dessert_name.map(dict_cholesterol)
    df_entree_side_dessert['sodium'] = entree_name.map(dict_sodium) + side_name.map(dict_sodium) + dessert_name.map(dict_sodium)
    df_entree_side_dessert['tag'] = entree_name.map(dict_tag) + ', ' + side_name.map(dict_tag) + ', ' + dessert_name.map(dict_tag)
    df_entree_side_dessert['brand_name'] = entree_name.map(dict_brand_name)
    
df_entree_side_dessert['meal_type'] = 'entree_side_dessert'
df_entree_side_dessert

,entree,side,dessert,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,tag,brand_name,meal_type
0,Chicken Selects (3 Piece),Small French Fries,Strawberry & Creme Pie,910.0,48.0,86.0,15.0,30.0,14.0,85.0,1040.0,"none, none, none",McDonald's,entree_side_dessert
1,Chicken Selects (3 Piece),Small French Fries,"Chocolate Shake, Medium",1240.0,51.0,131.0,90.0,41.0,16.0,140.0,1160.0,"none, none, none",McDonald's,entree_side_dessert
2,Chicken Selects (3 Piece),Small French Fries,"Arctic Orange Shake, Small",1120.0,47.0,116.0,74.0,37.0,14.0,125.0,1010.0,"none, none, none",McDonald's,entree_side_dessert
3,Chicken Selects (3 Piece),Small French Fries,Hot Fudge Sundae EOTF,930.0,42.0,93.0,47.0,35.0,11.0,100.0,1010.0,"none, none, none",McDonald's,entree_side_dessert
4,Chicken Selects (3 Piece),Small French Fries,Cherry and Creme Pie,900.0,49.0,84.0,14.0,30.0,14.0,85.0,915.0,"none, none, none",McDonald's,entree_side_dessert
5,Chicken Selects (3 Piece),Small French Fries,Marionberry Pie Ice Cream,880.0,48.0,81.0,17.0,31.0,14.0,110.0,945.0,"none, none, none",McDonald's,entree_side_dessert
6,Chicken Selects (3 Piece),Small French Fries,Udderly Chocolate Ice Cream,930.0,50.0,87.0,26.0,32.0,16.0,110.0,965.0,"none, none, none",McDonald's,entree_side_dessert
7,Chicken Selects (3 Piece),Small French Fries,Chocolate Coffee Cake,780.0,41.0,71.0,13.0,29.0,7.5,90.0,1030.0,"none, none, breakfast",McDonald's,entree_side_dessert
8,Chicken Selects (3 Piece),Small French Fries,Mixed Berry and Lemon Creme Pie,900.0,49.0,85.0,13.0,30.0,14.0,85.0,1020.0,"none, none, none",McDonald's,entree_side_dessert
9,Chicken Selects (3 Piece),Small French Fries,Cinnamon Coffee Cake,770.0,40.0,72.0,13.0,29.0,7.0,90.0,975.0,"none, none, breakfast",McDonald's,entree_side_dessert


In [57]:
# Add entree-side-dessert combinations that meet nutritional goals to df_meal
within_goal_entree_side_dessert_0 = (df_entree_side_dessert['calories'] < df_goals.loc[0,'calories']) & (df_entree_side_dessert['carbs'] < df_goals.loc[0,'carbs']) & (df_entree_side_dessert['total_fat'] < df_goals.loc[0,'total_fat']) & (df_entree_side_dessert['sugar'] < df_goals.loc[0,'sugar'])
df_meal_0 = df_meal_0.append(df_entree_side_dessert.loc[within_goal_entree_side_dessert_0])
df_meal_0 = df_meal_0[['brand_name',"entree", "side", "dessert", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_dessert_1 = (df_entree_side_dessert['calories'] < df_goals.loc[1,'calories']) & (df_entree_side_dessert['carbs'] < df_goals.loc[1,'carbs']) & (df_entree_side_dessert['total_fat'] < df_goals.loc[1,'total_fat']) & (df_entree_side_dessert['sugar'] < df_goals.loc[1,'sugar'])
df_meal_1 = df_meal_1.append(df_entree_side_dessert.loc[within_goal_entree_side_dessert_1])
df_meal_1 = df_meal_1[['brand_name',"entree", "side", "dessert", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_dessert_2 = (df_entree_side_dessert['calories'] < df_goals.loc[2,'calories']) & (df_entree_side_dessert['carbs'] < df_goals.loc[2,'carbs']) & (df_entree_side_dessert['total_fat'] < df_goals.loc[2,'total_fat']) & (df_entree_side_dessert['sugar'] < df_goals.loc[2,'sugar'])
df_meal_2 = df_meal_2.append(df_entree_side_dessert.loc[within_goal_entree_side_dessert_2])
df_meal_2 = df_meal_2[['brand_name',"entree", "side", "dessert", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

within_goal_entree_side_dessert_3 = (df_entree_side_dessert['calories'] < df_goals.loc[3,'calories']) & (df_entree_side_dessert['carbs'] < df_goals.loc[3,'carbs']) & (df_entree_side_dessert['total_fat'] < df_goals.loc[3,'total_fat']) & (df_entree_side_dessert['sugar'] < df_goals.loc[3,'sugar'])
df_meal_3 = df_meal_3.append(df_entree_side_dessert.loc[within_goal_entree_side_dessert_3])
df_meal_3 = df_meal_3[['brand_name',"entree", "side", "dessert", "calories", "total_fat", "carbs", "sugar", "protein", "saturated_fat", "cholesterol", "sodium", "meal_type","tag"]]

df_meal_0

,brand_name,entree,side,dessert,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,meal_type,tag
22,McDonald's,Egg McMuffin,NaN,NaN,300.0,12.0,30.0,3.0,18.0,6.0,245.0,730.0,entree_only,breakfast
24,McDonald's,Grilled Onion Cheddar Burger,NaN,NaN,310.0,13.0,32.0,6.0,15.0,6.0,45.0,640.0,entree_only,none
34,McDonald's,McDouble,NaN,NaN,380.0,18.0,34.0,7.0,23.0,8.0,70.0,840.0,entree_only,none
38,McDonald's,"Ranch Snack Wrap, Grilled",NaN,NaN,300.0,13.0,25.0,2.0,21.0,4.5,65.0,890.0,entree_only,none
45,McDonald's,"Premium McWrap Chicken & Ranch, Grilled",NaN,NaN,490.0,19.0,41.0,5.0,39.0,7.0,120.0,1510.0,entree_only,none
49,McDonald's,Buffalo Ranch McChicken,NaN,NaN,360.0,16.0,40.0,5.0,15.0,3.0,40.0,800.0,entree_only,none
66,McDonald's,Country Ham Biscuit,NaN,NaN,340.0,14.0,36.0,2.0,15.0,8.0,35.0,1600.0,entree_only,breakfast
67,McDonald's,Biscuit Chicken Sausage,NaN,NaN,350.0,17.0,36.0,2.0,13.0,8.0,50.0,1110.0,entree_only,breakfast
84,McDonald's,Pico Guacamole Grilled Chicken Sandwich on Potato Roll,NaN,NaN,470.0,17.0,40.0,6.0,38.0,5.0,115.0,1520.0,entree_only,none
125,McDonald's,Steak & Egg Burrito,NaN,NaN,350.0,15.0,38.0,3.0,16.0,6.0,115.0,950.0,entree_only,breakfast


In [58]:
# Fix tags
df_meal_0['tag'] = df_meal_0['tag'].apply(lambda x: re.sub(r"none, |, none", "", x))
df_meal_1['tag'] = df_meal_1['tag'].apply(lambda x: re.sub(r"none, |, none", "", x))
df_meal_2['tag'] = df_meal_2['tag'].apply(lambda x: re.sub(r"none, |, none", "", x))
df_meal_3['tag'] = df_meal_3['tag'].apply(lambda x: re.sub(r"none, |, none", "", x))

df_meal_0['tag'] = df_meal_0['tag'].apply(lambda x: re.sub(r"breakfast, breakfast", "breakfast", x))
df_meal_1['tag'] = df_meal_1['tag'].apply(lambda x: re.sub(r"breakfast, breakfast", "breakfast", x))
df_meal_2['tag'] = df_meal_2['tag'].apply(lambda x: re.sub(r"breakfast, breakfast", "breakfast", x))
df_meal_3['tag'] = df_meal_3['tag'].apply(lambda x: re.sub(r"breakfast, breakfast", "breakfast", x))

In [59]:
df_meal_3['brand_name'].unique()

array(["McDonald's", "Wendy's", 'Burger King', 'Taco Bell'], dtype=object)

In [61]:
df_meal_3[df_meal_3['tag'].str.contains('soda, soda')]

,brand_name,entree,side,dessert,calories,total_fat,carbs,sugar,protein,saturated_fat,cholesterol,sodium,meal_type,tag


### Export meal combinations to CSV

In [62]:
df_meal_0.to_csv('meal_0.csv')
df_meal_1.to_csv('meal_1.csv')
df_meal_2.to_csv('meal_2.csv')
df_meal_3.to_csv('meal_3.csv')

In [63]:
df_goal_daily.to_csv('goal_daily.csv')
df_goals.to_csv('goal_meal.csv')